# Mavlink Camera Base
> Mavlink experiments

[https://mavlink.io/en/mavgen_python/](https://mavlink.io/en/mavgen_python/)
[https://www.ardusub.com/developers/pymavlink.html](https://www.ardusub.com/developers/pymavlink.html)

https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_TYPE



In [ ]:
#| default_exp mavlink.cam_base

In [ ]:
import UAV.logging
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

Mavlink Routing
[https://ardupilot.org/dev/docs/mavlink-routing-in-ardupilot.html](https://ardupilot.org/dev/docs/mavlink-routing-in-ardupilot.html)
Each message contains a System ID and Component ID field to specify where the message came from. In addition some messages (including SET_POSITION_TARGET_GLOBAL_INT) include target_system and target_component fields to allow specifying which system/component should execute the command.
![](images/mavlink-routing.png){fig-align="center"}

[https://mavlink.io/en/guide/routing.html](https://mavlink.io/en/guide/routing.html) 
A MAVLINK network is made up of systems (vehicles, ground stations, antenna trackers, etc.), which may be composed from one or more components (autopilot, camera, servos, etc.).

Each system has a network-unique system id, and each component has a system-unique component id that can be used for addressing/routing:

The system id has a value between 1 and 255.
The default autopilot system id is usually 1. Users should allocate unique increasing id values when adding new autopilots to a network.
GCS systems and developer APIs typically use an ID at the top of the numeric range to reduce ID clashes (e.g. 255). Often their system ID is configurable to allow multi-GCS systems.
The component id is allocated by type and number from MAV_COMPONENT.
Messages can be intended for all systems, specific systems, all components in a system, or specific components within a system. The protocol defines two 8-bit fields that can (optionally) be specified in the message payload to indicate where the message should be sent/routed. If the ids are omitted or set to zero then the message is considered a broadcast (intended for all systems/components).

target_system: System that should execute the command
target_component: Component that should execute the command (requires target_system).
MAVLink components are expected to process messages that have a matching system/component id and broadcast messages. They are expected to route/resend messages that are intended for other (or all) recipients to other active channels (i.e. MAVLink systems may be connected across different transports, connected by a MAVLink system that routes the messages). Broadcast messages are forwarded to all channels that haven't seen the message. Addressed messages are resent on a new channel iff the system has previously seen a message from the target on that channel (messages are not resent if the addressee is not known or is on the original/incoming channel).

***Wireshark Filter***
`mavlink_proto.sysid!=255 && not icmp`
`mavlink_proto.sysid!=255 && mavlink_proto.sysid!=1 && not icmp` 

In [ ]:
#| export
import logging
logging.getLogger("uav").setLevel(logging.INFO)
logging.root.setLevel(logging.INFO)
import cv2

import numpy as np
from imutils import resize

import threading
import time
from pymavlink import mavutil
from pymavlink.dialects.v20 import common as mavlink2
import threading

import UAV.params as params
from UAV.imports import *   # TODO why is this relative import on nbdev_export?
import UAV
# gi.require_version('Gst', '1.0')

In [ ]:
logging.root.level

20

In [ ]:
logging.getLogger("uav").setLevel(logging.INFO)
logging.getLogger("uav").level
# UAV.logging.get_log_level()


20

In [ ]:
# #| export
# logging.basicConfig(format='%(asctime)-8s,%(msecs)-3d %(levelname)5s [%(filename)10s:%(lineno)3d] %(message)s',
#                     datefmt='%H:%M:%S',
#                     level=params.LOGGING_LEVEL)  # Todo add this to params
# logger = logging.getLogger(params.LOGGING_NAME)

In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
SYSTEM_GCS_CLIENT_ID = 195
SYSTEM_CAM_ID = 2   # 1 = vehicle, 195 = companion computer but here i use 2 for debugging
class MavLinkBase:
    """
    Mavlink Camera Base 
    """
    def __init__(self, connection_string, # "udpin:localhost:14550"
                 baudrate=57600, #baud rate of the serial port
                 mav_type=mavutil.mavlink.MAV_TYPE_CAMERA, # type used in heartbeat
                 is_server=True, # server or client
                 debug=False, # logging level
                 ):
        self._log = logging.getLogger("uav.{}".format(self.__class__.__name__))
        if debug:
            log_level = logging.DEBUG
        else:
            log_level = logging.INFO
        self._log.setLevel(log_level)
        self.connection_string = connection_string
        self.baudrate = baudrate
        # self.gstpipes = gstpipes
        # self.mav_type = mav_type
        
        self.init_extras(mav_type, is_server)  # Needs to be implemented in child class
        self.start_mavlink()
    

    def init_extras(self, mav_type, is_server):
        """ This is called from init() and can be used to set up additional parameters
            It models a mavlink Client. Can  be implemented in child class
            copy and change these as needed, consider call super().init_extras() first 
            https://mavlink.io/en/messages/common.html#MAV_COMPONENT
            """
        self.mav_type=mav_type  
        if is_server:
            self.log.info(f"init_extras() for Server, mav_type: {mav_type}") 
            self.source_system=SYSTEM_CAM_ID # source system   1 = vehicle, 195 = companion computer
            self.source_component=mavutil.mavlink.MAV_COMP_ID_CAMERA # source component https://mavlink.io/en/messages/common.html#MAV_COMPONENT
            self.target_system=SYSTEM_GCS_CLIENT_ID # source system   1 = vehicle, 195 = companion computer
            self.target_component=mavutil.mavlink.MAV_COMP_ID_USER1 # target component
            # self.target_component=0 # target component
            self.do_heartbeat=True   # send heartbeat
            self.do_listen=True   # listen for commands
            self.do_ack = True   # send ack

        else:  # client
            self.log.info(f"init_extras() for Client, mav_type: {mav_type}")   
            self.source_system=SYSTEM_GCS_CLIENT_ID # source system   1 = vehicle, 195 = companion computer
            self.source_component=mavutil.mavlink.MAV_COMP_ID_USER1 # source component
            self.target_system=SYSTEM_CAM_ID # target system   1 = vehicle, 195 = companion computer
            self.target_component=mavutil.mavlink.MAV_COMP_ID_CAMERA # target component
            # self.target_component=0 # target component
            self.do_heartbeat=True   # send heartbeat
            self.do_listen=False   # listen for commands
            self.do_ack = False   # send ack

    def start_mavlink(self):
        """Start the MAVLink connection."""
        # Create the connection  Todo add source_system and component options
        self.log.info("Starting MAVLink connection...")

        self.master = mavutil.mavlink_connection(self.connection_string, # "udpin:localhost:14550"
                                                 baud=self.baudrate, # baud rate of the serial port
                                                 source_system=int(self.source_system), # source system
                                                 source_component=int(self.source_component), # source component
                                                 dialect="ardupilotmega", # dialect
                                                 )
        self.master.target_system = self.target_system
        self.master.target_component = self.target_component
        # self.log.info(f"Mavlink Dialect: {self.master.dialect}")
        self.log.info(f"Source system: {self.master.source_system}, Source component: {self.master.source_component}")
        self.log.info(f"Target system: {self.master.target_system}, Target component: {self.master.target_component}")
        self.log.info(f"see https://mavlink.io/en/messages/common.html#MAV_COMPONENT")
         
        if self.do_heartbeat:
            self._t_heartbeat = threading.Thread(target=self.send_heartbeat, daemon=True)
            self._t_heartbeat.start()
        if self.do_listen:
            self._t_mav_listen = threading.Thread(target=self.listen, daemon=True)
            self._t_mav_listen.start()
            
    def __str__(self) -> str:
        return self.__class__.__name__

    def __repr__(self) -> str:
        return "<{}>".format(self)

    @property
    def log(self) -> logging.Logger:
        return self._log
    
    def send_heartbeat(self):
        """Send a heartbeat message to indicate the server is alive."""
        self._t_heartbeat_stop = False
        self.log.info("Starting heartbeat")
        while not self._t_heartbeat_stop:
            self.master.mav.heartbeat_send(
                self.mav_type,  # type
                # mavutil.mavlink.MAV_TYPE_ONBOARD_CONTROLLER,
                mavutil.mavlink.MAV_AUTOPILOT_INVALID,  # autopilot
                0,  # base_mode
                0,  # custom_mode
                mavutil.mavlink.MAV_STATE_ACTIVE,  # system_status
                3  # MAVLink version
            )
            # print("Cam heartbeat_send")
            time.sleep(1)  # Send every second

    def send_ack(self, msg):
        """Send an ACK message to indicate a command was received."""
        self.log.debug(f"Sending ACK target_system:{self.target_system} target_component:{self.target_component}")
        self.master.mav.command_ack_send(
                    msg.command,
                    mavutil.mavlink.MAV_RESULT_ACCEPTED,  # or other MAV_RESULT enum
                    # todo enabling these causes QGC not to show them
                    0,  # progress
                    0,  # result_param2
                    self.target_system,  # target_system
                    self.target_component,  # target_component

        )
        

    def listen(self):
        """Listen for MAVLink commands and trigger the camera when needed."""
        self._t_mav_listen_stop = False
        self.log.info("Listening for MAVLink commands...")
        while not self._t_mav_listen_stop:
            # Wait for a MAVLink message
            try:
                msg = self.master.recv_match(blocking=True, timeout=1)
                if msg is None:
                    continue
                if msg.get_type() == 'COMMAND_LONG':
                    self.log.debug(f"Received message {msg.command}")
                    if self.do_ack:
                        self.send_ack(msg)
                        
                if msg.get_type() == 'COMMAND_ACK':
                    self.log.debug(f" func listen() Received ACK ")
                # elif msg.get_type() == 'HEARTBEAT':
                #     self.log.debug(f"Received HEARTBEAT ")
                # elif msg.get_type() == 'STATUSTEXT':
                #     self.log.debug(f"Received STATUSTEXT ")
                # elif msg.get_type() == 'CAMERA_IMAGE_CAPTURED':
                #     self.log.debug(f"Received CAMERA_IMAGE_CAPTURED ")
                # elif msg.get_type() != 'COMMAND_LONG':
                #     self.log.debug(f"Received message ")
  
            except Exception as e:
                self.log.error(e)
                continue
            # msg = self.master.recv_match(timeout=1)
            if not msg:
                continue
                
        self.log.info("Stopped")
        
    def close(self):
        print(f"Closing {self.__class__.__name__}...")
        self.master.close()
        self.master.port.close()
        # for pipe in self.gstpipes:
        #     pipe.shutdown()
        # self.gstpipes.close()
        self._t_mav_listen_stop = True
        self._t_heartbeat_stop = True
        try:
            self._t_heartbeat.join()
            self._t_mav_listen.join()
        except Exception as e:
            pass

        self.log.info(f"{self.__class__.__name__}  closed")
    
    def __enter__(self):
        """ Context manager entry point for with statement."""
        return self # This value is assigned to the variable after 'as' in the 'with' statement
    
    def __exit__(self, exc_type, exc_value, traceback):
        """Context manager exit point."""
        self.close()
        return False  # re-raise any exceptions
#         
with MavLinkBase("udpout:localhost:14550", debug=True, is_server=True) as server:
    
    pass
time.sleep(0.2)
# 
# with MavLinkBase("udpout:localhost:14550", debug=True, is_server=False) as client:
#     pass
    

INFO   | uav.MavLinkBase      | 20:08:26.036 |[638785952.py: 37] MainThread | init_extras() for Server, mav_type: 30
INFO   | uav.MavLinkBase      | 20:08:26.036 |[638785952.py: 61] MainThread | Starting MAVLink connection...
INFO   | uav.MavLinkBase      | 20:08:26.037 |[638785952.py: 72] MainThread | Source system: 2, Source component: 100
INFO   | uav.MavLinkBase      | 20:08:26.037 |[638785952.py: 73] MainThread | Target system: 195, Target component: 25
INFO   | uav.MavLinkBase      | 20:08:26.038 |[638785952.py: 74] MainThread | see https://mavlink.io/en/messages/common.html#MAV_COMPONENT
INFO   | uav.MavLinkBase      | 20:08:26.038 |[638785952.py: 96] Thread-70 (send_heartbeat) | Starting heartbeat
INFO   | uav.MavLinkBase      | 20:08:26.038 |[638785952.py:128] Thread-71 (listen) | Listening for MAVLink commands...
INFO   | uav.MavLinkBase      | 20:08:26.039 |[638785952.py:158] Thread-71 (listen) | Stopped
INFO   | uav.MavLinkBase      | 20:08:26.039 |[638785952.py:175] MainTh

Closing MavLinkBase...


In [ ]:
class CamClient(MavLinkBase):
 
    def __init__(self, connection_string, # "udpin:localhost:14550"
                 baudrate=57600, #baud rate of the serial port
                 mav_type=mavutil.mavlink.MAV_TYPE_ONBOARD_CONTROLLER, # type used in heartbeat
                 debug=False, # logging level
                 ):    
        super(CamClient, self).__init__( connection_string = connection_string,
                baudrate=baudrate, #baud rate of the serial port
                mav_type=mav_type, # type used in heartbeat
                debug=debug, # logging level
                is_server=False,   # camera Client                      
                )
        # self.last_command_id = None

    def send_command(self, 
                     command_id: int, # mavutil.mavlink.MAV_CMD....
                     params: list,): # list of parameters
        self.log.debug(f"Sending command {command_id}...")

        self.last_command_id = command_id
        self.master.mav.command_long_send(
            self.master.target_system,  # target_system
            self.master.target_component,  # target_component
            command_id,  # command id
            0,  # confirmation
            *params  # command parameters
        )

    def listen_for_ack(self):
        # print("Waiting for ACK...")
        
        # while True:
            # msg = self.master.recv_match(blocking=True, timeout=1)
        msg = self.master.recv_match(type='COMMAND_ACK', blocking=True, timeout=1)
        if msg is None:
            self.log.debug("No Ack received")
            return
        # if ack_msg.get_type() == 'COMMAND_ACK':
        if msg.command == self.last_command_id:
            self.log.debug(f"Received ACK for command {msg.command}. Result: {mavutil.mavlink.enums['MAV_RESULT'][msg.result].name}")
            return


    def trigger_camera(self, camera_id=1):
        # Use MAV_CMD_DO_DIGICAM_CONTROL to trigger the camera
        self.send_command(mavutil.mavlink.MAV_CMD_DO_DIGICAM_CONTROL, 
                          [camera_id,  # param1 (session)  or cam # (0 for all cams)
                           1,  # param2 (trigger capture)
                           0,  # param3 (zoom pos)
                           0,  # param4 (zoom step)
                           0,  # param5 (focus lock)
                           0,  # param6 (shot ID)
                           0,  # param7 (command ID)
                          ])
        self.listen_for_ack()
        self.log.info("sent Camera trigger")
        
    def _trigger_camera(self, camera_id=1):
        # Use MAV_CMD_DO_DIGICAM_CONTROL to trigger the camera
        # self.log("Sending command...")
        # self.log.info(f"Target system: {self.master.target_system}, Target component: {self.master.target_component}")
        self.master.mav.command_long_send(
            self.master.target_system,  # target_system
            self.master.target_component,  # target_component
            mavutil.mavlink.MAV_CMD_DO_DIGICAM_CONTROL,  # command
            0,  # confirmation
            camera_id,  # param1 (session)  or cam # (0 for all cams)
            1,  # param2 (trigger capture)
            0,  # param3 (zoom pos)
            0,  # param4 (zoom step)
            0,  # param5 (focus lock)
            0,  # param6 (shot ID)
            0,  # param7 (command ID)
        )
        self.log.info("sent Camera trigger")
        
class CamServer(MavLinkBase):
    def __init__(self, connection_string, # "udpin:localhost:14550"
             baudrate=57600, #baud rate of the serial port
             mav_type=mavutil.mavlink.MAV_TYPE_CAMERA, # type used in heartbeat
             debug=False, # logging level
             ):    
        super(CamServer, self).__init__( connection_string = connection_string,
                baudrate=baudrate, #baud rate of the serial port
                mav_type=mav_type, # type used in heartbeat
                debug=debug, # logging level
                is_server=True,   # camera server  
                )


if True:
# with CamClient("udpout:localhost:14550", debug=true) as client1:

    with CamClient("udpout:localhost:14445", debug=true) as client2:

        with CamServer("udpin:localhost:14445", debug=true) as server:
            for i in range(5):
                # client1.trigger_camera(1)
                # time.sleep(1)
                client2.trigger_camera(2)
                time.sleep(1)
                # print("Still alive")


INFO   | uav.CamClient        | 20:09:51.486 |[638785952.py: 48] MainThread | init_extras() for Client, mav_type: 18
INFO   | uav.CamClient        | 20:09:51.487 |[638785952.py: 61] MainThread | Starting MAVLink connection...
INFO   | uav.CamClient        | 20:09:51.487 |[638785952.py: 72] MainThread | Source system: 195, Source component: 25
INFO   | uav.CamClient        | 20:09:51.488 |[638785952.py: 73] MainThread | Target system: 2, Target component: 100
INFO   | uav.CamClient        | 20:09:51.488 |[638785952.py: 74] MainThread | see https://mavlink.io/en/messages/common.html#MAV_COMPONENT
INFO   | uav.CamClient        | 20:09:51.488 |[638785952.py: 96] Thread-75 (send_heartbeat) | Starting heartbeat
INFO   | uav.CamServer        | 20:09:51.489 |[638785952.py: 37] MainThread | init_extras() for Server, mav_type: 30
INFO   | uav.CamServer        | 20:09:51.489 |[638785952.py: 61] MainThread | Starting MAVLink connection...
INFO   | uav.CamServer        | 20:09:51.490 |[638785952.py

Closing CamServer...


INFO   | uav.CamServer        | 20:09:57.513 |[638785952.py:175] MainThread | CamServer  closed


Closing CamClient...


INFO   | uav.CamClient        | 20:09:58.517 |[638785952.py:175] MainThread | CamClient  closed


Mavlink Camera Client

In [ ]:
test_eq(1,1)

In [ ]:
assert False, "Stop here"

AssertionError: Stop here

https://github.com/mavlink/MAVSDK/issues/1803

So I managed to change OpenHD in this regard.
No idea why I had such a hard time wrapping my head around, but now it works the following:
OpenHD binds port 127.0.0.1:14551 and listens on 127.0.0.1:14550
AND
instead of using sendto() with a unbound port (which then in turn means the sender port can be anything) messages are sent with sendto() from the bound port (the same that is used for listening).

So messages from OpenHD to mavsdk go the following:
OpenHD (out) via 127:0:0:1:14551 sent to 127:0:0:0:1:14550

So when mavsdk receives the first message, the sender address::port is 127:0:0:1:14551 and mavsdk can send the messages back to 127:0:0:1:14551.

https://julianoes.com/
The ports are not symmetrical! QGC listens on local port 14550 and sends UDP packets back to wherever messages came from.



In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()

In [ ]:
# #| export
# import time
# 
# class CommandSender:
#         def __init__(self, connection_string, # "udpin:localhost:14550"
#                  mav_type=mavutil.mavlink.MAV_TYPE_CAMERA, # type
#                  baudrate=57600, #baud rate of the serial port
#                  camera_id=0, # camera id
#                  gstpipes=[], # list of gstreamer pipelines to control
#                  debug=False, # logging level
#                 ):
#         super(CamClient, self).__init__( connection_string, # "udpin:localhost:14550"
#                  mav_type=mav_type, # type
#                  baudrate=baudrate, #baud rate of the serial port
#                  camera_id=camera_id, # camera id
#                  gstpipes=gstpipes, # list of gstreamer pipelines to control
#                  debug=debug, # logging level
#                  heartbeat=False,   # send heartbeat
#                  ack = False,   # send ack
#                 )
#     def __init__(self, connection_string):
#         # Create the connection
#         self.master = mavutil.mavlink_connection(connection_string)
#         self.last_command_id = None
# 
#     def send_command(self, command_id, params):
#         print(f"Sending command {command_id}...")
#         self.last_command_id = command_id
#         self.master.mav.command_long_send(
#             self.master.target_system,  # target_system
#             self.master.target_component,  # target_component
#             command_id,  # command id
#             0,  # confirmation
#             *params  # command parameters
#         )
# 
#     def listen_for_ack(self):
#         print("Waiting for ACK...")
#         
#         while True:
#             msg = self.master.recv_match(blocking=True)
#             
#             if msg.get_type() == 'COMMAND_ACK':
#                 if msg.command == self.last_command_id:
#                     print(f"Received ACK for command {msg.command}. Result: {mavutil.mavlink.enums['MAV_RESULT'][msg.result].name}")
#                     break

